# (done) import library & define env

In [1]:
# %pip install llama-index-vector-stores-qdrant
# %pip install --upgrade llama-index
# %pip install --upgrade fastembed
# %pip install --upgrade llama-index-storage-chat-store-redis
# %pip install --upgrade llama-index-core
# %pip install --upgrade redis
# %pip install --upgrade llama-index-storage-kvstore-redis
# %pip install "numpy<2.0"
# %pip install --upgrade pandas pyarrow llama-index
# %pip install "pybind11>=2.12"
# %pip install --upgrade llama-index-retrievers-bm25

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

In [3]:
import os
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

llm = OpenAI(
    model="gpt-4o-mini"
)

embed_model = OpenAIEmbedding(
    model="text-embedding-3-small"
)

Settings.llm = llm
Settings.embed_model = embed_model

# (done) read and parse

In [4]:
import os
import tempfile
import pdfplumber
from typing import List
from datetime import datetime
from llama_index.core import SimpleDirectoryReader

def process_single_pdf(pdf_path: str) -> List:
    """
    Load and parse a specific PDF file using SimpleDirectoryReader.
    Args:
        pdf_path (str): Path to the PDF file to process.

    Returns:
        List: Documents from SimpleDirectoryReader.
    """
    if not os.path.exists(pdf_path):
        raise FileNotFoundError(f"File not found: {pdf_path}")

    if not pdf_path.lower().endswith('.pdf'):
        raise ValueError(f"File {pdf_path} is not a PDF file")

    with tempfile.TemporaryDirectory() as temp_dir:
        try:
            pdf_filename = os.path.basename(pdf_path)
            temp_text_file = os.path.join(temp_dir, f'{os.path.splitext(pdf_filename)[0]}.txt')

            with pdfplumber.open(pdf_path) as pdf:
                text_parts = []
                for page in pdf.pages:
                    page_text = page.extract_text()
                    if page_text:
                        text_parts.append(page_text)

            with open(temp_text_file, 'w', encoding='utf-8') as f:
                f.write(''.join(text_parts))

            print(f"File processed: {pdf_filename}")

            documents = SimpleDirectoryReader(temp_dir).load_data()

            issue_date_str = input("Please enter the issue date (YYYY-MM-DD): ")
            try:
                issue_date = datetime.strptime(issue_date_str, "%Y-%m-%d")
                issue_date_str = issue_date.strftime("%Y-%m-%d")
            except ValueError:
                print("Invalid date format. Defaulting to today's date.")
                issue_date_str = datetime.today().strftime("%Y-%m-%d")

            outdated_input = input("Is this document outdated? (yes/no): ").strip().lower()
            outdated = True if outdated_input == "yes" else False

            for doc in documents:
                doc.metadata['issue_date'] = issue_date_str
                doc.metadata['outdated'] = outdated

        except Exception as e:
            raise Exception(f"Error processing file {pdf_filename}: {str(e)}")

    return documents

input_dir = "TULD02.pdf"
documents = process_single_pdf(input_dir)

File processed: TULD02.pdf
Invalid date format. Defaulting to today's date.


# (done) define qdrant & docstore

In [5]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams

qdrant_client = QdrantClient(url="http://localhost:6333")
collection_name = "documents_collection"

collections = qdrant_client.get_collections()
print("Current collections:", collections)

if collection_name not in [collection.name for collection in collections.collections]:
    vector_params = VectorParams(size=1536, distance="Cosine")
    qdrant_client.create_collection(
        collection_name=collection_name,
        vectors_config=vector_params
    )
    print(f"Collection '{collection_name}' đã được tạo.")
else:
    print(f"Collection '{collection_name}' đã tồn tại.")

Current collections: collections=[]
Collection 'documents_collection' đã được tạo.


In [ ]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core.storage.docstore import SimpleDocumentStore

vector_store = QdrantVectorStore(
    client=qdrant_client, 
    collection_name="documents_collection"
)
docstore = SimpleDocumentStore()

# (done) process
- indexing
- chunking
- pipeline (save to qdrant & save to docstore)

In [ ]:
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.extractors import QuestionsAnsweredExtractor, KeywordExtractor
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.core import Settings
import nest_asyncio


nest_asyncio.apply()
"""Processes a set of documents through an ingestion pipeline, extracts semantic and keyword information,
and stores the processed nodes in a vector database.

Steps:
1. Splits the input documents into semantic chunks using `SemanticSplitterNodeParser`.
2. Extracts questions and keywords from the chunks using `QuestionsAnsweredExtractor` and `KeywordExtractor`.
3. Stores the processed nodes in a vector database (e.g., Qdrant).

Args:
    documents (List[Document]): The input documents to be processed. Each document is typically 
        an instance of a class containing text and optional metadata.

    vector_store (VectorStore): The vector store where processed nodes will be stored. This could 
        be a Qdrant vector store or similar.

Returns:
    List[Node]: A list of processed nodes containing semantic chunks, extracted keywords, 
    and associated metadata.

Raises:
    Exception: If the pipeline encounters issues during document processing or saving to the vector store."""

splitter = SemanticSplitterNodeParser(
    buffer_size=1,
    breakpoint_percentile_threshold=95,
    embed_model=Settings.embed_model
)

extractors = [
    QuestionsAnsweredExtractor(llm=Settings.llm, questions=1),
    KeywordExtractor(llm=Settings.llm, keywords=5),
]

transformations = [splitter] + extractors + [embed_model]

pipeline = IngestionPipeline(
    transformations=transformations,
    vector_store=vector_store
)

# run pipeline
nodes = pipeline.run(documents=documents, show_progress=True, batch_size=64)


Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/122 [00:00<?, ?it/s]

100%|██████████| 7/7 [00:01<00:00,  4.93it/s]


Generating embeddings:   0%|          | 0/7 [00:00<?, ?it/s]

In [8]:
from llama_index.core.storage.docstore import SimpleDocumentStore

docstore = SimpleDocumentStore()
docstore.add_documents(nodes)

# (done) only bm25 (answer right)

In [9]:
from llama_index.retrievers.bm25 import BM25Retriever
import Stemmer

# We can pass in the index, docstore, or list of nodes to create the retriever
bm25_retriever = BM25Retriever.from_defaults(
    docstore=docstore,
    similarity_top_k=2,
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)

resource module not available on Windows


In [ ]:
from llama_index.core.response.notebook_utils import display_source_node

retrieved_nodes = bm25_retriever.retrieve(
    "Đối tượng thi hành"
)
for node in retrieved_nodes:
    display_source_node(node, source_length=5000)

**Node ID:** 33f9de68-f8ff-400e-8160-ff4b24282911<br>**Similarity:** 1.4459148645401<br>**Text:** 2. Đại diện tập thể người lao động:
Ông: NGUYỄN DANH TÙNG - Chủ tịch Công đoàn Tổng Công ty.
Hai bên cam kết thực hiện đúng các quy định của pháp luật và cùng nhau
thỏa thuận ký kết Thỏa ước lao động tập thể, gồm 03 chương và 16 điều;
CHƯƠNG I
NHỮNG QUY ĐỊNH CHUNG
Thoả ước lao động tập thể này quy định mối quan hệ lao động giữa tập thể
lao động và NSDLĐ về các điều kiện lao động và sử dụng lao động, quyền và
nghĩa vụ của mỗi bên trong thời hạn Thỏa ước có hiệu lực. Mọi trường hợp khác
trong mối quan hệ lao động không quy định trong bản Thỏa ước lao động tập thể
này, sẽ được giải quyết theo Bộ luật Lao động và các văn bản quy phạm pháp
luật hiện hành.
Điều 1. Đối tượng thi hành:
Thỏa ước lao động tập thể này áp dụng đối với:
- Người sử dụng lao động (NSDLĐ); Bao gồm Hội đồng Quản trị, Ban
Tổng giám đốc.
- Tập thể người lao động (NLĐ): Bao gồm người lao động đang làm việc
tại Tổng công ty, kể cả người lao động trong thời gian học nghề, thử việc, NLĐ
vào làm việc sau ngày Thỏa ước có hiệu lực.
- Ban Chấp hành công đoàn Tổng công ty.<br>

**Node ID:** e96e9eb5-ec17-4176-860b-602794cc8cf7<br>**Similarity:** 1.2385129928588867<br>**Text:** Một số thỏa thuận khác.
Tùy theo hiệu quả sản xuất kinh doanh hàng năm và phần lợi nhuận được
trích để lại các quỹ cho Tổng công ty. Hội đồng Quản trị, Ban Tổng giám đốc
Tổng công ty sẽ thống nhất với BCH công đoàn cơ sở các nội dung như sau:
- Chi thăm hỏi cho cán bộ hưu trí; NLĐ, thân nhân là thương binh, liệt sỹ,
gia đình có công cách mạng vào dịp 27 tháng 7 và tết nguyên đán…
- Tổ chức tặng quà động viên con NLĐ học đạt từ loại giỏi trở lên nhân
dịp khai trường; thiếu niên, nhi đồng nhân các ngày quốc tế thiếu nhi, tết trung
thu.
- Thăm hỏi gia đình NLĐ; (bị bệnh hiểm nghèo hoặc khi có hữu sự).
- Trợ cấp đột xuất NLĐ khi gặp sự cố rủi ro: Thiên tai, hỏa hoạn, bệnh
hiểm nghèo, tai nạn lao động, tai nạn giao thông mất sức lao động từ 31% trở
lên.
- Chi phí cho các hoạt động văn hóa, thể thao theo chương trình kế hoạch
hàng năm do các đoàn thể đề xuất được thống nhất của Ban Tổng giám đốc
Tổng công ty.
- Có chính sách đối với nguồn nhân lực có trình độ và chính sách thu hút
nguồn lao động đáp ứng sự nghiệp phát triển công ty trong từng giai đoạn.
- Có chính sách đối với người có đóng góp, hỗ trợ hoạt động Tổng công ty
đem lại hiệu quả cao.
- Tùy theo nguồn quỹ phúc lợi tập thể hiện còn hàng năm, BCH Công
đoàn Tổng công ty đề xuất với Hội đồng Quản trị, Ban Tổng giám đốc xem xét
tổ chức cho các đối tượng được khen thưởng, có sáng kiến kinh nghiệm được
xét và công nhận loại A, có nhiều đóng góp tích cực cho Tổ chức Đảng bộ, Tổng
công ty, Tổ chức Công đoàn và Đoàn cơ sở Tổng công ty đi thăm quan, du lịch;
địa điểm thời gian sẽ do lãnh đạo Quyết định
Điều 13. Tranh chấp lao động
Hai bên thực hiện theo quy định tại chương XIV Bộ luật Lao động.
CHƯƠNG III: ĐIỀU KHOẢN THI HÀNH
Điều 14. Trách nhiệm thi hành Thỏa ước:
- NSDLĐ, Ban chấp hành CĐCS có trách nhiệm triển khai, quán triệt đến
tất cả NLĐ trong Tổng Công ty biết và thực hiện đúng các nội dung đã thỏa
thuận trong bản Thỏa ước này.
- Trong quá trình thực hiện Thỏa ước lao động tập thể, NSDLĐ và BCH
CĐCS cùng nhau xem xét kiểm tra việc thực hiện Thỏa ước, nếu có nội dung
chưa phù hợp thì một trong hai bên đề nghị điều chỉnh theo qui định.
5Điều 15. Giải quyết tranh chấp liên quan đến các thỏa thuận trong Thỏa
ước.
- Các tranh chấp liên quan đến nội dung thỏa ước là tranh chấp lao động và
được giải quyết theo quy định của pháp luật lao động. Mọi tranh chấp (cá nhân,
tập thể), NLĐ đều phải gửi yêu cầu bằng văn bản đến Ban chấp hành CĐCS
hoặc người có thẩm quyền (Ban Tổng giám đốc) để được xem xét giải quyết.
- Khi có tranh chấp phát sinh trong quan hệ lao động, các bên tranh chấp thì
BCH Công đoàn và người có thẩm quyền giải quyết tranh chấp sử dụng thỏa
ước này và các quy định được thừa nhận trong bản thỏa ước này làm cơ sở pháp
lý để xem xét vụ việc.
Điều 16.<br>

# (done) QueryFusionRetriever (answer right)

In [ ]:
from llama_index.core import VectorStoreIndex,StorageContext

storage_context = StorageContext.from_defaults(
    docstore=docstore,
    vector_store=vector_store
)

index = VectorStoreIndex(nodes=nodes,vector_store=vector_store)

In [ ]:
import nest_asyncio

nest_asyncio.apply()

from llama_index.core.retrievers import QueryFusionRetriever
from llama_index.retrievers.bm25 import BM25Retriever


retriever = QueryFusionRetriever(
    # call 2 type retrieve
    [
        index.as_retriever(similarity_top_k=2),
        BM25Retriever.from_defaults(
            docstore=index.docstore, similarity_top_k=2
        ),
    ],
    # max query
    num_queries=2,
    use_async=True,
)

In [21]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine(retriever)

response = query_engine.query("Đối tượng thi hành")
print(response)

Đối tượng thi hành Thỏa ước lao động tập thể này bao gồm:

- Người sử dụng lao động (NSDLĐ), bao gồm Hội đồng Quản trị và Ban Tổng giám đốc.
- Tập thể người lao động (NLĐ), bao gồm người lao động đang làm việc tại Tổng công ty, kể cả người lao động trong thời gian học nghề, thử việc, và NLĐ vào làm việc sau ngày Thỏa ước có hiệu lực.
- Ban Chấp hành công đoàn Tổng công ty.


# (still fixing) chat engine

In [34]:
from llama_index.core.postprocessor import SentenceEmbeddingOptimizer
from llama_index.core.postprocessor import EmbeddingRecencyPostprocessor
from llama_index.core.postprocessor import LLMRerank

node_postprocessors = [
    SentenceEmbeddingOptimizer(
        embed_model=Settings.embed_model,
        threshold_cutoff=0.7,
    ),
    EmbeddingRecencyPostprocessor(date_key="date", similarity_cutoff=0.7),
    LLMRerank(top_n=2),
]

In [40]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import get_response_synthesizer
from llama_index.core.response_synthesizers.type import ResponseMode

response_synthesizer = get_response_synthesizer(llm=Settings.llm, response_mode=ResponseMode.COMPACT)

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=node_postprocessors,
)

In [41]:
response = query_engine.query("Đối tượng thi hành")
print(response)

ValueError: Optimizer returned zero sentences.

In [37]:
from llama_index.core import PromptTemplate
from llama_index.core.llms import ChatMessage, MessageRole

CUSTOM_PROMPT = PromptTemplate(
    """
    Based on the conversation history between the User and the Assistant, along with the User's new question, analyze and understand the question within the context of the conversation.
    Provide a relevant response in Vietnamese, using a professional tone like a Human Resource Specialist.  

    <Conversation History>
    {chat_history}

    <Current Question>
    {question}
    """
)

custom_chat_history = [
    ChatMessage(
        role=MessageRole.USER,
        content="Hello assistant, we are having a conversation about the company's regulations.",
    ),
    ChatMessage(
        role=MessageRole.ASSISTANT,
        content="Great, would you like to know more information about the company's regulations?",
    ),
]

In [38]:
from llama_index.core.chat_engine import CondenseQuestionChatEngine

chat_engine = CondenseQuestionChatEngine.from_defaults(
    query_engine=query_engine,
    condense_question_prompt=CUSTOM_PROMPT,
    chat_history=custom_chat_history,
    verbose=False
)

In [39]:
query = "cho tôi biết Đối tượng thi hành là ai"
response = chat_engine.chat(query)

ValueError: Optimizer returned zero sentences.

In [33]:
print(response)

Mọi cá nhân làm việc trong công ty, bao gồm nhân viên, quản lý và các bên liên quan khác, đều có trách nhiệm tuân thủ các quy định đã được ban hành. Điều này đảm bảo rằng tất cả các cấp bậc trong tổ chức đều thực hiện đúng các nội dung trong thỏa ước lao động và các quy định liên quan. Nếu cần thêm thông tin chi tiết về từng đối tượng cụ thể hoặc các quy định liên quan, hãy cho biết để có thể cung cấp thêm.
